# How to use RawKernels in CuPy
**Author:** [Severin Dicks](https://github.com/Intron7)

Lets look at a CUDA kernel to Normalize a CSR Matrix to a Targetsum

In [1]:
import scanpy as sc
import anndata
import cupy as cp

import time
import rapids_singlecell as rsc
import math

import warnings
warnings.filterwarnings("ignore")

In [3]:
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator
rmm.reinitialize(
    managed_memory=False, # Allows oversubscription
    pool_allocator=True, # default is False
)
cp.cuda.set_allocator(rmm_cupy_allocator)

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [4]:
data_load_start = time.time()

In [5]:
%%time
adata = sc.read("../2024_gpu_severin.dicks/notebooks/h5/norm_work.h5ad")

CPU times: user 1.17 s, sys: 6.9 s, total: 8.07 s
Wall time: 12.3 s


In [6]:
%%time
rsc.get.anndata_to_GPU(adata)

CPU times: user 587 ms, sys: 2.4 s, total: 2.99 s
Wall time: 4 s


## Lets look at X

X is a sparse CSR matrix. 

In [7]:
X = adata.X.copy()

In [8]:
X

In [9]:
X.indptr

array([         0,       1807,       3056, ..., 1005501520, 1005503246,
       1005505079], dtype=int32)

In [10]:
X.indices

array([   11,    25,    32, ..., 21842, 21844, 21856], dtype=int32)

In [11]:
X.data

array([ 1.,  1.,  1., ...,  2., 14.,  2.], dtype=float32)

## Lets Write/ Explore the Kernel

For each row we sum up all `Values` to get the `Rowsum`.

Than we divide the `Targetsum` by the `Rowsum` to get the `scaler` for each row.

Lastly we multiply the `Values` with the rowwise `scaler`.

In [15]:
from cuml.common.kernel_utils import cuda_kernel_factory

_mul_kernel_csr = r"""
extern "C" __global__
void mul_csr(const int *indptr, float *data,
                    int nrows, int tsum) {
        int row = blockDim.x * blockIdx.x + threadIdx.x;

        if(row >= nrows)
            return;

        float scale = 0.0;
        int start_idx = indptr[row];
        int stop_idx = indptr[row+1];

        for(int i = start_idx; i < stop_idx; i++)
            scale += data[i];

        if(scale > 0.0) {
            scale = tsum / scale;
            for(int i = start_idx; i < stop_idx; i++)
                data[i] *= scale;
        }
    }
"""

mul_kernel = cp.RawKernel(_mul_kernel_csr, "mul_csr")

## Lets Call the Kernel

As we see before the kernel only give use the function for 1 row. But we need to normalize all rows. So the function call contains the grid and thread layout.

We need one thread per row. Thats how we have to create the launch.

In [16]:
mul_kernel(
    (math.ceil(X.shape[0] / 128),), #How many blocks do we need
    (128,), # the number of threads per block
    (X.indptr, X.data, X.shape[0], int(10000)),# Giving the function the arguments
)

In [17]:
 X.data

array([ 2.471577 ,  2.471577 ,  2.471577 , ...,  4.8134775, 33.694344 ,
        4.8134775], dtype=float32)